I have listed most (maybe not all) constraints on the system.  This is based on the idea that users can initialize the database with 3 files (a routes file, a driver file, and a driver assignment file).  However, once these files are loaded, the user cannot append to the data that we are storing through an additional csv file.  For example, if a user initialized the DB with ./routes_file_1, they cannot also append to the routes with a ./routes_file_2.  This is important for a number of reasons that I don't want to write out but we can talk about them if we want.  We could allow for a "clear DB" button and allow the user to re-initialize though.

# Use dask or sframe

In [14]:
import pandas as pd

df = pd.read_csv('./test_csvs/assignments/Lin_Assignment.csv')
# del df['Unnamed: 0']
df.to_csv('./test_csvs/assignments/Lin_Assignment.csv') 
'./test_csvs/assignments/'
df

,Unnamed: 0,100A,1,M
0,0,100B,1,T
1,1,100C,1,W
2,2,100A,1,U
3,3,100B,1,F
4,4,100C,1,S
5,5,100A,1,s
6,6,200A,2,M
7,7,100A,2,T
8,8,100B,2,W
9,9,100C,2,U


In [1]:
import multiprocessing
n_cpus = multiprocessing.cpu_count()

In [11]:
import dask.dataframe as dd
import dask

assignment_csv_path = './test_csvs/assignments/Lin_Assignment.csv'

df = dask.dataframe.read_csv(assignment_csv_path, dtype='str')
# df = dd.read_csv(assignment_csv_path, dtype='str')
df.head(26)

,Unnamed: 0,100A,1,M
0,0,100B,1,T
1,1,100C,1,W
2,2,100A,1,U
3,3,100B,1,F
4,4,100C,1,S
5,5,100A,1,s
6,6,200A,2,M
7,7,100A,2,T
8,8,100B,2,W
9,9,100C,2,U
